# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [15]:
import sys
import os
from pathlib import Path

# Obtém o diretório atual do notebook
current_dir = Path.cwd()

# Adiciona o diretório raiz do projeto ao Python path
project_root = current_dir.parent
sys.path.append(str(project_root))

# Agora você pode importar de src.utils
from src.utils import get_azure_openai_config
import openai

azure_config = get_azure_openai_config()

config_list = [{
    "model": azure_config["deployment_name"],
    "api_key": azure_config["api_key"],
    # "api_base": azure_config["api_base"],
    "api_type": "azure",
    "api_version": azure_config["api_version"]
}]

llm_config = {"config_list": config_list}

## Define an AutoGen agent

In [16]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [17]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Why did the scarecrow win an award? 

Because he was outstanding in his field!


In [18]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Of course! Please provide the joke you'd like me to repeat.


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [19]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [20]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Absolutely, Joe! You know what they say: laughter is like Wi-Fi—sometimes you just need to reboot to connect! So, what’s been your favorite thing to joke about lately?

--------------------------------------------------------------------------------
joe (to cathy):

Well, Cathy, you know what they say: "You can’t teach an old dog new tricks!" But have you seen how many new ways they’ve figured out to destroy a couch? Seriously, if they put as much effort into learning as they do into digging holes, I'd have a PhD in canine behavior by now!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that’s so true! Old dogs are basically professional couch assassins. It’s like they have a secret society of home destruction! "Meeting at 8, chew the couch, dig in the garden, and accide

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [21]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'name': 'joe',
  'role': 'assistant'},
 {'content': 'Absolutely, Joe! You know what they say: laughter is like '
             'Wi-Fi—sometimes you just need to reboot to connect! So, what’s '
             'been your favorite thing to joke about lately?',
  'name': 'cathy',
  'role': 'user'},
 {'content': 'Well, Cathy, you know what they say: "You can’t teach an old dog '
             'new tricks!" But have you seen how many new ways they’ve figured '
             'out to destroy a couch? Seriously, if they put as much effort '
             "into learning as they do into digging holes, I'd have a PhD in "
             'canine behavior by now!',
  'name': 'joe',
  'role': 'assistant'},
 {'content': 'Haha, that’s so true! Old dogs are basically professional couch '
             'assassins. It’s like they have a secret society of home '
             'destruction! "Meeting at 8, chew the couch, dig in the garden, '
            

In [22]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-4o-mini': {'completion_tokens': 188,
                                                      'cost': 0.00015494999999999997,
                                                      'prompt_tokens': 281,
                                                      'total_tokens': 469},
                                      'total_cost': 0.00015494999999999997},
 'usage_including_cached_inference': {'gpt-4o-mini': {'completion_tokens': 188,
                                                      'cost': 0.00015494999999999997,
                                                      'prompt_tokens': 281,
                                                      'total_tokens': 469},
                                      'total_cost': 0.00015494999999999997}}


In [23]:
pprint.pprint(chat_result.summary)

('Haha, that’s so true! Old dogs are basically professional couch assassins. '
 'It’s like they have a secret society of home destruction! "Meeting at 8, '
 'chew the couch, dig in the garden, and accidentally throw up in the shoe." '
 'And I swear, they think they’re ninjas! You turn your back for one second, '
 'and suddenly, your favorite chair is a pile of fluff and dignity!')


## Get a better summary of the conversation

In [24]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Absolutely, Joe! You know what they say: laughter is like Wi-Fi—sometimes you just need to reboot to connect! So, what’s been your favorite thing to joke about lately?

--------------------------------------------------------------------------------
joe (to cathy):

Well, Cathy, you know what they say: "You can’t teach an old dog new tricks!" But have you seen how many new ways they’ve figured out to destroy a couch? Seriously, if they put as much effort into learning as they do into digging holes, I'd have a PhD in canine behavior by now!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that’s so true! Old dogs are basically professional couch assassins. It’s like they have a secret society of home destruction! "Meeting at 8, chew the couch, dig in the garden, and accide

In [25]:
pprint.pprint(chat_result.summary)

('Joe and Cathy joke about the antics of dogs, particularly how they can '
 'destroy furniture and cause messes while seeming to do so with great skill. '
 'They highlight the humorous contrast between a dog’s destructive behavior '
 'and its ability to learn new tricks.')


## Chat Termination

Chat can be terminated using a termination conditions.

In [26]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [27]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Alright, let’s keep it rolling. Why don’t scientists trust atoms? Because they make up everything! What’s your favorite type of joke?

--------------------------------------------------------------------------------
joe (to cathy):

Hey Cathy! That's a classic! I love a good pun, but you can't beat a solid one-liner. Like, "I told my wife she was drawing her eyebrows too high. She looked surprised!" What about you? Got any favorites?

--------------------------------------------------------------------------------
cathy (to joe):

Oh, I love a good one-liner! Here's one: "I used to play piano by ear, but now I use my hands!" What's your go-to one-liner for a laugh?

--------------------------------------------------------------------------------
joe (to cathy):

That's a good one! My go-to? "I'm on a whiskey diet. I'v

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}

In [ ]:
cathy.send(message="What's last joke we talked about?", recipient=joe)